<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E9%9D%92%E7%A9%BA%E6%96%87%E5%BA%AB%E3%81%AE%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 青空文庫のテキストデータを取得する (3)

*   履歴
    *   前処理を追加しました（2025/10/14）
    *   利用するテキストデータを globis-university/aozorabunko-clean から取得するように変更しました（2025/10/22）

*   「▶」ボタンで最初のコードセルを実行してください。表が表示されたら，表の右上の「Filter」ボタンを押して絞り込み検索をして目的の作品Noを見つけてください。


In [ ]:
#@title 青空文庫の作家別作品リストを読み込む

from pathlib import Path
import pandas as pd
from google.colab import data_table

if not Path('list_person_all_utf8.csv').exists():
  !curl -RL https://www.aozora.gr.jp/index_pages/list_person_all_utf8.zip | funzip > list_person_all_utf8.csv

df_list_person = pd.read_csv('list_person_all_utf8.csv')

data_table.DataTable(df_list_person, num_rows_per_page=10)

*   目的の作品IDが分かったら，下のフォームの「作品ID」欄に入力してください。

In [ ]:
#@title 作品IDを指定してファイルを取得する

def aozora_download_text(work_id, output_path='book.txt'):
  import json
  from datasets import load_dataset

  # データセットを取得する（取得したデータはキャッシュされる）
  ds = load_dataset('globis-university/aozorabunko-clean')

  # 作品IDで絞り込む
  ds_filtered = ds.filter(lambda row: int(row['meta']['作品ID'], base=10) == work_id)

  # ヒットした件数を確認する
  assert ds_filtered['train'].num_rows > 0, "❌ 該当する作品IDは見つかりませんでした"
  if ds_filtered['train'].num_rows > 1:
    print("⚠️ 同じ作品IDを持つ複数の作品が見つかりました")

  # 作品に関する情報を取得
  book = ds_filtered['train'][0]

  # 取得したテキストを保存する（本文のみ）
  with open(output_path, "w", encoding="utf-8") as f:
    f.write(book['text'])
  print(f"✅ {output_path} に保存しました")

  # テキストファイルを開く
  from google.colab import files
  files.view(output_path)

  # 自動的にダウンロードする
  files.download(output_path)


作品ID = 789  #@param {"type":"integer"}
出力ファイル名 = 'book.txt'  #@param {"type":"string"}

aozora_download_text(作品ID, 出力ファイル名)